In [1]:
# Changed code relative to others:
# encoding no longer use cos sin (no noise)
# dump log now dump both pickle and txt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from scipy import stats
from tqdm import tqdm_notebook

import Config
from Config import *
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC


In [4]:
# train data once. This is without retraining so beware!
def onepass_train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_acc = hdc.fit_once(traindata, trainlabels, param)
    test_acc = hdc.test(testdata, testlabels)
    print("Train: %f \t \t Test: %f"%(train_acc, test_acc))
    return train_acc, test_acc

In [5]:
# train data. with retrain
def train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_accs = []
    test_accs = []
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_accs.append(hdc.fit(traindata, trainlabels, param))
        test_accs.append(hdc.test(testdata, testlabels, param["kernel"]))
        if len(train_accs) % 5 == 1:
            print("Train: %f \t \t Test: %f"%(train_accs[-1], test_accs[-1]))
        if train_accs[-1] == 1:
            print("Train: %f \t \t Test: %f"%(train_accs[-1], test_accs[-1]))
            break
    return np.asarray(train_accs), np.asarray(test_accs)

In [6]:
# train data once via batch
def train_batch(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config, batch = 50):
    train_accs = []
    for i in range(len(traindata)//batch):
        train_accs.append(hdc.fit_sep(traindata[i*batch: (i+1)*batch], trainlabels[i*batch: (i+1)*batch], param))
    train_acc = np.mean(np.asarray(train_accs))
    test_acc = hdc.test(testdata, testlabels, param["kernel"])
    #print("Train: %f \t \t Test: %f"%(train_acc, test_acc))
    return train_acc, test_acc

In [7]:
# train data via batch, with retrain
def batched_train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config, batch = 50):
    train_accs = []
    test_accs = []
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_acc, test_acc = train_batch(hdc, traindata, trainlabels, testdata, testlabels, param, batch)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
        if len(train_accs) % 5 == 1:
            print("Train: %f \t \t Test: %f"%(train_accs[-1], test_accs[-1]))
        if train_accs[-1] == 1:
            print("Train: %f \t \t Test: %f"%(train_accs[-1], test_accs[-1]))
            break
    return np.asarray(train_accs), np.asarray(test_accs)

In [8]:
def dump_log(param, train_acc, test_acc, filename):
    joblib.dump((param, train_acc, test_acc), open(filename+".pkl", "wb"), compress=True)
    file = open(filename+".txt", "w")
    file.write("Max train: %.2f \tMax test: %.2f \n"%(train_acc*100, test_acc*100))
    file.write(str(param))
    file.close()

In [9]:
dl = DL.Dataloader()
nFeatures, nClasses, traindata, trainlabels, testdata, testlabels = dl.getParam()


Loading dataset UCIHAR from UCIHAR
Loading train data... train data of shape (6213, 561) loaded
Loading test data...  test  data of shape (1554, 561) loaded
Data Loaded. Num of features = 561 Num of Classes = 12

In [10]:
#Data shuffling 
shuf_train = np.random.permutation(len(traindata))
traindata = traindata[shuf_train]
trainlabels = trainlabels[shuf_train]

shuf_test = np.random.permutation(len(testdata))
testdata = testdata[shuf_test]
testlabels = testlabels[shuf_test]

In [11]:
traindata = traindata[:50000]
testdata = testdata[:10000]

In [12]:
param = Config.config
param["nFeatures"] = nFeatures
param["nClasses"] = nClasses
print(param)

{'data_location': '../dataset/', 'directory': 'UCIHAR', 'dataset': 'UCIHAR', 'D': 2000, 'vector': 'Gaussian', 'mu': 0, 'sigma': 1, 'binarize': 0, 'lr': 1, 'sparse': 0, 's': 0.1, 'binaryModel': 0, 'checkpoints': False, 'kernel': <Kernel_T.COS: 1>, 'width': None, 'height': None, 'nLayers': 5, 'uniform_dim': 1, 'uniform_ker': 1, 'dArr': None, 'k': 3, 'kArr': None, 'one_shot': 0, 'data_percentages': [1.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 'train_percent': 1, 'dropout': 0, 'drop_percentages': [0, 0.1, 0.2, 0.5], 'dropout_rate': 0, 'update_type': <Update_T.FULL: 1>, 'iter_per_trial': 3, 'iter_per_encoding': 5, 'epochs': 40, 'nFeatures': 561, 'nClasses': 12}


In [13]:
################# VANILLA #################
hdb = HDB.HD_basis(Config.Generator_T.Vanilla, param)
basis = hdb.getBasis()
bid = hdb.getParam()["id"]
# Update param with bid
param = hdb.getParam()
print(bid)

Generating vanilla HD basis of shape... 


7700


(2000, 561)
Encoding time: 0.07970571517944336 


In [14]:
# Retrieve info upto basis generator, given correct bid
#bid = 6679
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)

In [15]:
hde = HDE.HD_encoder(basis)

trainencoded = hde.encodeData(traindata)
#HDE.saveEncoded(trainencoded, trainlabels, bid, "train")

testencoded = hde.encodeData(testdata)
#HDE.saveEncoded(testencoded, testlabels, bid, "test")

Encoding data of shape (6213, 561)


Time spent: 2 sec
Encoding data of shape (1554, 561)


Time spent: 0 sec


In [16]:
# Retrieve info upto encoder, given correct bid
#bid = 6679
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)
#trainencoded, trainlabels = HDE.loadEncoded("encoded_%d_train.pkl"%bid)
#testencoded, testlabels = HDE.loadEncoded("encoded_%d_test.pkl"%bid)

# Onepass - train

In [17]:
# Should have 90%
one_hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = onepass_train(one_hdc, trainencoded, trainlabels, testencoded, testlabels)

Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.DOT: 0>)] 


Train: -1.000000 	 	 Test: 0.308600


In [18]:
accs = dict()
for k in Kernel_T:
    accs[k]= one_hdc.test(testencoded, testlabels, k)

KeyboardInterrupt: 

In [ ]:
for k in Kernel_T:
    print(k, accs[k])

# Onepass - train - automation

In [14]:
def onepass_automation(param):
    
    hdb = HDB.HD_basis(Config.Generator_T.Vanilla, param)
    basis = hdb.getBasis()
    bid = hdb.getParam()["id"]
    param = hdb.getParam() # Update param with bid
    
    hde = HDE.HD_encoder(basis)
    trainencoded = hde.encodeData(traindata)
    testencoded = hde.encodeData(testdata)
    
    # Should have 90%
    one_hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
    train_acc, test_acc = onepass_train(one_hdc, trainencoded, trainlabels, testencoded, testlabels)
    
    accs = dict()
    for k in Kernel_T:
        accs[k]= one_hdc.test(testencoded, testlabels, k)
        print(k, accs[k])
    return accs

In [15]:
accs_D = dict()
for D in [500, 1000, 2000, 3000, 4000]:
    param["D"] = D
    accs_D[D] = onepass_automation(param)

Generating vanilla HD basis of shape... 

(500, 21)
Encoding time: 0.030895233154296875 
Encoding data of shape (1913, 21)


Time spent: 0 sec
Encoding data of shape (213, 21)


Time spent: 0 sec
Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.COS: 1>)] 



Train: -1.000000 	 	 Test: 0.924883
Kernel_T.DOT 0.9248826291079812
Kernel_T.COS 0.8591549295774648
Kernel_T.PL1 0.9014084507042254
Kernel_T.PL3 0.9248826291079812
Kernel_T.PL4 0.6619718309859155
Kernel_T.BT1 0.9248826291079812
Kernel_T.BT3 0.892018779342723
Kernel_T.BT4 0.8873239436619719


...

(4000, 21)
Encoding time: 0.04886984825134277 
Encoding data of shape (1913, 21)


Time spent: 0 sec
Encoding data of shape (213, 21)



Train: -1.000000 	 	 Test: 0.924883
Kernel_T.DOT 0.9248826291079812
Kernel_T.COS 0.8544600938967136
Kernel_T.PL1 0.8309859154929577
Kernel_T.PL3 0.9061032863849765
Kernel_T.PL4 0.7981220657276995


Time spent: 0 sec
Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.COS: 1>)] 


Kernel_T.BT1 0.8685446009389671
Kernel_T.BT3 0.8497652582159625
Kernel_T.BT4 0.8544600938967136


In [16]:
for D in accs_D.keys():
    print(D)
    for k in accs_D[D]:
        print(accs_D[D][k])

500
0.9248826291079812
0.8591549295774648
0.9014084507042254
0.9248826291079812
0.6619718309859155
0.9248826291079812
0.892018779342723
0.8873239436619719
1000
0.9248826291079812
0.8591549295774648
0.8685446009389671
0.8591549295774648
0.8591549295774648
0.8779342723004695
0.8591549295774648
0.8685446009389671
2000
0.9248826291079812
0.8544600938967136
0.8169014084507042
0.9014084507042254
0.7981220657276995
0.9014084507042254
0.8450704225352113
0.8497652582159625
3000
0.9248826291079812
0.8450704225352113
0.812206572769953
0.92018779342723
0.7605633802816901
0.8215962441314554
0.8450704225352113
0.8450704225352113
4000
0.9248826291079812
0.8544600938967136
0.8309859154929577
0.9061032863849765
0.7981220657276995
0.8685446009389671
0.8497652582159625
0.8544600938967136


# Retrain-nonl automation 

In [13]:
def retrainnl_automation(param):
    
    hdb = HDB.HD_basis(Config.Generator_T.Vanilla, param)
    basis = hdb.getBasis()
    bid = hdb.getParam()["id"]
    param = hdb.getParam() # Update param with bid
    
    hde = HDE.HD_encoder(basis)
    trainencoded = hde.encodeData(traindata)
    testencoded = hde.encodeData(testdata)
    
    # Should have 90%
    retrain_hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
    train_acc, test_acc = train(retrain_hdc, trainencoded, trainlabels, testencoded, testlabels)
    
    accs = dict()
    for k in Kernel_T:
        accs[k]= retrain_hdc.test(testencoded, testlabels, k)
        print(k, accs[k])
    return accs

In [14]:
accs_D = dict()
for D in [500, 1000, 2000, 3000, 4000]:
    param["D"] = D
    accs_D[D] = retrainnl_automation(param)

Generating vanilla HD basis of shape... 

(500, 561)
Encoding time: 0.03688383102416992 
Encoding data of shape (6213, 561)


Time spent: 0 sec
Encoding data of shape (1554, 561)


Time spent: 0 sec
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.COS: 1>)] 


...

(4000, 561)
Encoding time: 0.14459609985351562 
Encoding data of shape (6213, 561)


Time spent: 4 sec
Encoding data of shape (1554, 561)


Time spent: 1 sec


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.COS: 1>)] 


Train: 0.721230 	 	 Test: 0.840412
Train: 0.911798 	 	 Test: 0.888031
Train: 0.942701 	 	 Test: 0.958172
Train: 0.945115 	 	 Test: 0.951737
Train: 0.953163 	 	 Test: 0.911840
Train: 0.950909 	 	 Test: 0.918919
Train: 0.973765 	 	 Test: 0.950450
Train: 0.962659 	 	 Test: 0.960103

Kernel_T.DOT 0.7966537966537967
Kernel_T.COS 0.9607464607464608
Kernel_T.PL1 0.767052767052767
Kernel_T.PL3 0.8146718146718147
Kernel_T.PL4 0.8262548262548263
Kernel_T.BT1 0.8043758043758044
Kernel_T.BT3 0.8886743886743886
Kernel_T.BT4 0.8822393822393823


In [15]:
for D in accs_D.keys():
    print(D)
    for k in accs_D[D]:
        print(accs_D[D][k])
        #print(k)

500
0.7947232947232947
0.9594594594594594
0.21171171171171171
0.35006435006435005
0.3680823680823681
0.2696267696267696
0.6801801801801802
0.694980694980695
1000
0.7155727155727156
0.9066924066924067
0.8712998712998713
0.6621621621621622
0.6866151866151866
0.6814671814671814
0.8970398970398971
0.907979407979408
2000
0.7966537966537967
0.9465894465894465
0.4195624195624196
0.7741312741312741
0.7767052767052767
0.788931788931789
0.9144144144144144
0.9266409266409267
3000
0.7760617760617761
0.9665379665379665
0.696911196911197
0.7258687258687259
0.750965250965251
0.7715572715572716
0.8056628056628057
0.8262548262548263
4000
0.7966537966537967
0.9607464607464608
0.767052767052767
0.8146718146718147
0.8262548262548263
0.8043758043758044
0.8886743886743886
0.8822393822393823


# Test with error automation

In [13]:
def test_with_error_automation(param, traindata, trainlabels, testdata, testlabels):
    
    global lvls, rates
    
    hdb = HDB.HD_basis(Config.Generator_T.Vanilla, param)
    basis = hdb.getBasis()
    bid = hdb.getParam()["id"]
    param = hdb.getParam() # Update param with bid
    
    hde = HDE.HD_encoder(basis)
    trainencoded = hde.encodeData(traindata)
    testencoded = hde.encodeData(testdata)
    
    # Should have 90%
    retrain_hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
    train_acc, test_acc = train(retrain_hdc, trainencoded, trainlabels, testencoded, testlabels)
    
    accs = dict()
    for err_lvl in lvls:
        for err_rate in rates:
            for k in [Kernel_T.BT4]: #[Kernel_T.BT3, Kernel_T.BT4]:
                accs[(err_lvl, err_rate, k)]= retrain_hdc.test_with_error(testencoded, testlabels, err_rate, err_lvl, k)
                print(err_lvl, err_rate, k, accs[(err_lvl, err_rate, k)])
    return accs

In [14]:
#Ds = [500, 1000, 2000, 3000]
#lvls = [1, 2, 3, 4]
#rates = [0, 0.01, 0.02, 0.05, 0.10, 0.20, 0.30]

Ds = [1000]
lvls = [1, 3, 5, 7]
rates = [0.5 + i*2/100 for i in reversed(range(26))]

accs_D = dict()
for D in Ds:
    param["D"] = D
    accs_D[D] = test_with_error_automation(param, traindata, trainlabels, testdata, testlabels)

Generating vanilla HD basis of shape... 

(1000, 561)
Encoding time: 0.05742812156677246 
Encoding data of shape (6213, 561)


Time spent: 0 sec
Encoding data of shape (1554, 561)


Time spent: 0 sec
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.COS: 1>)] 


...

7 0.58 Kernel_T.BT4 0.5958815958815958
Class given:
 [15 11  4  3  8  3  0  7 14  6  8  3 15 12 12 15 11  7  5  2  5  9 15  4
  5 14 15  7 11  9  8  9  7  9  7 15 13  2  8  1  6 11 12  8  0  3 11  7
 10  1 11  0 11  2 15 14 10 11  2 10  7  1  8 11 15 10  9  0 10  4  6 10
 13 12  2 11  0 14 10 11  9 14  9  1  8 13  5  9  0  9  9  0  9  7 12  6
  7 10  5 13 11  4  7  0 12  8  1 10  5  8  8  7  5 15  0  8 10 14 15  7
  5  5  3  6  8  7 12  4  7 14 11 11 10  4  0  9 12  9 10  5  8  8  6 15
  4  9  2 11  9 13  6  9 10  3  8 15 10  1 13  7 10  6 15 14  0  9  5 12
  2  5  4  3 10  9  4  4  9 12  2  1  5 12  7  5  7 11 11  8 15  8  8  7
 10  1  9  2  4 10  8  6 10  1 14  9  2  5  7  0 10  0  9  8  5 11  7  3
  3 12  4  4  7 14  3  7 11  5  0  0  3 11  6 13  6 10 15 11  7  0 12  5
 13 12  7 11 11 14  4  1 10  3 13  9  8  4 12  8  8  0 13  1  8 11  8  2
 11  5  7  7  0  6  8  5  0  0  7  9 11 10  4 14  9 12  7  0  6 15 10  4
 12  7  0 15  3  7  6 15 10 14  9  3  6  3  2  8 13  1  7  6  7  9  0  

7 0.56 Kernel_T.BT4 0.6866151866151866
Class given:
 [15 11  4  3  8  3  0  7 14  6  8  3 15 12 12 15 11  7  5  2  5  9 15  4
  5 14 15  7 11  9  8  9  7  9  7 15 13  2  8  1  6 11 12  8  0  3 11  7
 10  1 11  0 11  2 15 14 10 11  2 10  7  1  8 11 15 10  9  0 10  4  6 10
 13 12  2 11  0 14 10 11  9 14  9  1  8 13  5  9  0  9  9  0  9  7 12  6
  7 10  5 13 11  4  7  0 12  8  1 10  5  8  8  7  5 15  0  8 10 14 15  7
  5  5  3  6  8  7 12  4  7 14 11 11 10  4  0  9 12  9 10  5  8  8  6 15
  4  9  2 11  9 13  6  9 10  3  8 15 10  1 13  7 10  6 15 14  0  9  5 12
  2  5  4  3 10  9  4  4  9 12  2  1  5 12  7  5  7 11 11  8 15  8  8  7
 10  1  9  2  4 10  8  6 10  1 14  9  2  5  7  0 10  0  9  8  5 11  7  3
  3 12  4  4  7 14  3  7 11  5  0  0  3 11  6 13  6 10 15 11  7  0 12  5
 13 12  7 11 11 14  4  1 10  3 13  9  8  4 12  8  8  0 13  1  8 11  8  2
 11  5  7  7  0  6  8  5  0  0  7  9 11 10  4 14  9 12  7  0  6 15 10  4
 12  7  0 15  3  7  6 15 10 14  9  3  6  3  2  8 13  1  7  6  7  9  0  

7 0.54 Kernel_T.BT4 0.7213642213642214
Class given:
 [15 11  4  3  8  3  0  7 14  6  8  3 15 12 12 15 11  7  5  2  5  9 15  4
  5 14 15  7 11  9  8  9  7  9  7 15 13  2  8  1  6 11 12  8  0  3 11  7
 10  1 11  0 11  2 15 14 10 11  2 10  7  1  8 11 15 10  9  0 10  4  6 10
 13 12  2 11  0 14 10 11  9 14  9  1  8 13  5  9  0  9  9  0  9  7 12  6
  7 10  5 13 11  4  7  0 12  8  1 10  5  8  8  7  5 15  0  8 10 14 15  7
  5  5  3  6  8  7 12  4  7 14 11 11 10  4  0  9 12  9 10  5  8  8  6 15
  4  9  2 11  9 13  6  9 10  3  8 15 10  1 13  7 10  6 15 14  0  9  5 12
  2  5  4  3 10  9  4  4  9 12  2  1  5 12  7  5  7 11 11  8 15  8  8  7
 10  1  9  2  4 10  8  6 10  1 14  9  2  5  7  0 10  0  9  8  5 11  7  3
  3 12  4  4  7 14  3  7 11  5  0  0  3 11  6 13  6 10 15 11  7  0 12  5
 13 12  7 11 11 14  4  1 10  3 13  9  8  4 12  8  8  0 13  1  8 11  8  2
 11  5  7  7  0  6  8  5  0  0  7  9 11 10  4 14  9 12  7  0  6 15 10  4
 12  7  0 15  3  7  6 15 10 14  9  3  6  3  2  8 13  1  7  6  7  9  0  

7 0.52 Kernel_T.BT4 0.5958815958815958
Class given:
 [15 11  4  3  8  3  0  7 14  6  8  3 15 12 12 15 11  7  5  2  5  9 15  4
  5 14 15  7 11  9  8  9  7  9  7 15 13  2  8  1  6 11 12  8  0  3 11  7
 10  1 11  0 11  2 15 14 10 11  2 10  7  1  8 11 15 10  9  0 10  4  6 10
 13 12  2 11  0 14 10 11  9 14  9  1  8 13  5  9  0  9  9  0  9  7 12  6
  7 10  5 13 11  4  7  0 12  8  1 10  5  8  8  7  5 15  0  8 10 14 15  7
  5  5  3  6  8  7 12  4  7 14 11 11 10  4  0  9 12  9 10  5  8  8  6 15
  4  9  2 11  9 13  6  9 10  3  8 15 10  1 13  7 10  6 15 14  0  9  5 12
  2  5  4  3 10  9  4  4  9 12  2  1  5 12  7  5  7 11 11  8 15  8  8  7
 10  1  9  2  4 10  8  6 10  1 14  9  2  5  7  0 10  0  9  8  5 11  7  3
  3 12  4  4  7 14  3  7 11  5  0  0  3 11  6 13  6 10 15 11  7  0 12  5
 13 12  7 11 11 14  4  1 10  3 13  9  8  4 12  8  8  0 13  1  8 11  8  2
 11  5  7  7  0  6  8  5  0  0  7  9 11 10  4 14  9 12  7  0  6 15 10  4
 12  7  0 15  3  7  6 15 10 14  9  3  6  3  2  8 13  1  7  6  7  9  0  

7 0.5 Kernel_T.BT4 0.694980694980695


In [15]:
for D in Ds:
    for cfg in reversed(list(accs_D[D].keys())):
        if cfg[0] == 1 and cfg[2] == Kernel_T.BT4:
            #print(D, cfg, accs_D[D][cfg] + 0.09)
            print(accs_D[D][cfg] - 0.05 )#- accs_D[D][])

In [16]:
for rate in reversed(rates):
    print(rate)

0.5
0.52
0.54
0.56
0.58
0.6
0.62
0.64
0.66
0.6799999999999999
0.7
0.72
0.74
0.76
0.78
0.8
0.8200000000000001
0.8400000000000001
0.86
0.88
0.9
0.9199999999999999
0.94
0.96
0.98
1.0


# Comparing test results - retrain

In [32]:
hdcs = dict()
accs = dict()

In [18]:

param["kernel"] = k = Kernel_T.DOT
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.DOT: 0>)] 


Train: 0.676200 	 	 Test: 0.869600
Train: 0.925100 	 	 Test: 0.935400
Train: 0.962800 	 	 Test: 0.887600
Train: 0.959800 	 	 Test: 0.950200
Train: 0.982100 	 	 Test: 0.943400
Train: 0.985800 	 	 Test: 0.905000
Train: 0.992400 	 	 Test: 0.937200
Train: 0.989700 	 	 Test: 0.952000
Train: 0.958600 	 	 Test: 0.951800
Train: 0.995000 	 	 Test: 0.943400
Train: 0.997700 	 	 Test: 0.955400
Train: 0.998400 	 	 Test: 0.955200
Train: 0.997900 	 	 Test: 0.955400
Train: 0.998700 	 	 Test: 0.939400
Train: 0.998500 	 	 Test: 0.934800
Train: 0.997800 	 	 Test: 0.956800
Train: 0.999200 	 	 Test: 0.955800
Train: 0.999000 	 	 Test: 0.955800
Train: 0.998800 	 	 Test: 0.956000
Train: 1.000000 	 	 Test: 0.955400
Train: 1.000000 	 	 Test: 0.955400


In [19]:

param["kernel"] = k = Kernel_T.PL1
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.PL1: 2>)] 


Train: 0.598100 	 	 Test: 0.853000
Train: 0.844400 	 	 Test: 0.662000
Train: 0.849800 	 	 Test: 0.830400
Train: 0.879000 	 	 Test: 0.913400
Train: 0.869700 	 	 Test: 0.876000
Train: 0.854600 	 	 Test: 0.756600
Train: 0.901500 	 	 Test: 0.896800
Train: 0.852600 	 	 Test: 0.888400
Train: 0.904900 	 	 Test: 0.883400
Train: 0.852400 	 	 Test: 0.911000



In [20]:

param["kernel"] = k = Kernel_T.PL3
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.PL3: 3>)] 


Train: 0.803300 	 	 Test: 0.872600
Train: 0.962400 	 	 Test: 0.884600
Train: 0.978500 	 	 Test: 0.940000
Train: 0.973300 	 	 Test: 0.944600
Train: 0.992100 	 	 Test: 0.949000
Train: 0.993200 	 	 Test: 0.948400
Train: 0.996800 	 	 Test: 0.948400
Train: 0.997700 	 	 Test: 0.951600
Train: 0.998300 	 	 Test: 0.948800
Train: 0.995900 	 	 Test: 0.944200



In [21]:

param["kernel"] = k = Kernel_T.PL4
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.PL4: 4>)] 


Train: 0.790800 	 	 Test: 0.888200
Train: 0.969200 	 	 Test: 0.934200
Train: 0.985700 	 	 Test: 0.935400
Train: 0.990000 	 	 Test: 0.947800
Train: 0.992500 	 	 Test: 0.946600
Train: 0.989000 	 	 Test: 0.942200
Train: 0.998300 	 	 Test: 0.942200
Train: 0.999400 	 	 Test: 0.949600
Train: 0.998200 	 	 Test: 0.951800
Train: 0.986300 	 	 Test: 0.949000



In [ ]:
param["kernel"] = k = Kernel_T.BT1
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)

# preprocess data every epoch is not every time economic
trainprocessed = hdc.preprocess_data(trainencoded, k)
testprocessed = hdc.preprocess_data(testencoded, k)

train_acc, test_acc = batched_train(hdc, trainprocessed, trainlabels, testprocessed, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037), ('kernel', <Kernel_T.BT1: 5>)] 


Train: 0.795500 	 	 Test: 0.847000
Train: 0.918200 	 	 Test: 0.897400
Train: 0.935900 	 	 Test: 0.902400
Train: 0.936700 	 	 Test: 0.724400
Train: 0.938300 	 	 Test: 0.897000
Train: 0.937400 	 	 Test: 0.893600
Train: 0.947300 	 	 Test: 0.904600
Train: 0.949300 	 	 Test: 0.899600
Train: 0.946700 	 	 Test: 0.889400
Train: 0.947900 	 	 Test: 0.886600
Train: 0.937000 	 	 Test: 0.896400


In [31]:
param["kernel"] = k = Kernel_T.BT3
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


AssertionError: 

In [ ]:
param["kernel"] = k = Kernel_T.BT4
hdcs[k] = hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
accs[k] = hdc.test(testencoded, testlabels, k)

# Retrained - automation

In [20]:
def retrain_type(param, kernel_t, trainencoded, trainlabels, testencoded, testlabels):
    
    param["kernel"] = k = kernel_t
    hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)

    # preprocess data every epoch is not every time economic
    # trainprocessed = hdc.preprocess_data(trainencoded, k)
    # testprocessed = hdc.preprocess_data(testencoded, k)

    train_acc, test_acc = batched_train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
    
    return max(test_acc), hdc.test(testencoded, testlabels, k)

In [25]:
def retrain_automation(param):
    
    hdb = HDB.HD_basis(Config.Generator_T.Vanilla, param)
    basis = hdb.getBasis()
    bid = hdb.getParam()["id"]
    param = hdb.getParam() # Update param with bid
    
    hde = HDE.HD_encoder(basis)
    trainencoded = hde.encodeData(traindata)
    testencoded = hde.encodeData(testdata)
    
    accs = dict()
    for k in [Kernel_T.DOT,Kernel_T.COS, Kernel_T.PL1, Kernel_T.PL3, Kernel_T.PL4]:#[Kernel_T.BT1, Kernel_T.BT3, Kernel_T.BT4]: # Kernel_T: #
        accs[k] = retrain_type(param, k, trainencoded, trainlabels, testencoded, testlabels)
        print(k, accs[k])
    return accs

In [26]:
accs_D = dict()
for D in [1000, 2000]:
    param["D"] = D
    accs_D[D] = retrain_automation(param)

Generating vanilla HD basis of shape... 

(1000, 115)
Encoding time: 0.04495573043823242 
Encoding data of shape (50000, 115)


Time spent: 4 sec
Encoding data of shape (8927, 115)


Time spent: 0 sec
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.DOT: 0>)] 


...

Train: 0.976320 	 	 Test: 0.990030
Train: 0.981580 	 	 Test: 0.989582
Train: 0.982220 	 	 Test: 0.990030
Train: 0.983060 	 	 Test: 0.997200

Kernel_T.PL1 (0.9973115268287218, 0.9909264030469362)


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.PL3: 3>)] 


Train: 0.979900 	 	 Test: 0.990030
Train: 0.982820 	 	 Test: 0.990254
Train: 0.984360 	 	 Test: 0.990030
Train: 0.984200 	 	 Test: 0.990590

Kernel_T.PL3 (0.9976475859751316, 0.9976475859751316)


Sep Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 1), ('kernel', <Kernel_T.PL4: 4>)] 


Train: 0.978800 	 	 Test: 0.990366
Train: 0.983820 	 	 Test: 0.990366
Train: 0.984140 	 	 Test: 0.990030
Train: 0.984140 	 	 Test: 0.990366

Kernel_T.PL4 (0.9976475859751316, 0.9910384227624062)


In [29]:
for D in accs_D.keys():
    print(D)
    for k in accs_D[D]:
        x, y = accs_D[D][k]
        print(x)

1000
0.9974235465441917
0.997087487397782
0.996975467682312
0.9976475859751316
0.996975467682312
2000
0.9974235465441917
0.9974235465441917
0.9973115268287218
0.9976475859751316
0.9976475859751316


In [17]:
np.set_printoptions(threshold=sys.maxsize)